In [2]:
# demonstrate a simple computation graph

import torch
import torch.nn.functional as F
import torch

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([3.3], requires_grad=True)
z = w1 * x1 + b
a = torch.sigmoid(z)
loss = F.binary_cross_entropy(a, y)

grad_L_w1 = torch.autograd.grad(loss, w1, retain_graph=True)
grad_L_b = torch.autograd.grad(loss, b, retain_graph=True)
print(f'grad_L_w1: {grad_L_w1}')
print(f'grad_L_b: {grad_L_b}')

# all the gradients can be autocomputed using the backward method
loss.backward()
print(f'w1.grad: {w1.grad}')
print(f'b.grad: {b.grad}')

grad_L_w1: (tensor([-0.0036]),)
grad_L_b: (tensor([-0.0033]),)
w1.grad: tensor([-0.0036])
b.grad: tensor([-0.0033])


In [13]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(
            
            #1st hidden layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            # 2nd hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(20, num_outputs)
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits
    
model = NeuralNetwork(50, 3)
print(model)

# How many trainable parameters does the model have?
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_params} total parameters.')

print(model.layers[0].weight)
print(model.layers[0].weight.shape)

torch.manual_seed(123)
X = torch.rand(1, 50)

with torch.no_grad():
    out = torch.softmax(model(X), dim=1)

print(out)


NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)
2213 total parameters.
Parameter containing:
tensor([[ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        [-0.0920, -0.0480,  0.0105,  ..., -0.0923,  0.1201,  0.0330],
        ...,
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509],
        [-0.1250,  0.0513,  0.0366,  ..., -0.1370,  0.1074, -0.0704]],
       requires_grad=True)
torch.Size([30, 50])
tensor([[0.2801, 0.3635, 0.3565]])


In [17]:
# Understanding data loaders

X_train = torch.tensor(
    [
        [-1.2, 3.1],
        [-0.9, 2.9],
        [-0.5, 2.6],
        [2.3, 1.1],
        [2.7, -1.5]
    ]
)

y_train = torch.tensor([0, 0, 0, 1, 1])

X_test = torch.tensor(
    [
        [-1.1, 3.0],
        [2.4, 1.0]
    ]
)

y_test = torch.tensor([0, 1])

from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.label = y

    def __getitem__(self, index):
        return self.features[index], self.label[index]
    
    def __len__(self):
        return self.label.shape[0]
    
train_dataset = ToyDataset(X_train, y_train)
test_dataset = ToyDataset(X_test, y_test)

# Instantiate a data loader
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset = train_dataset,
    batch_size = 2,
    shuffle=True,
    num_workers=0
)

test_loader = DataLoader(
    dataset = test_dataset,
    batch_size = 2,
    shuffle=False,
    num_workers=0
)

# iterate over the training dataloader
for idx, (x,y) in enumerate(train_loader):
    print(f'batch {idx + 1}:')
    print(x)
    print(y)

batch 1:
tensor([[-0.5000,  2.6000],
        [ 2.3000,  1.1000]])
tensor([0, 1])
batch 2:
tensor([[-1.2000,  3.1000],
        [ 2.7000, -1.5000]])
tensor([0, 1])
batch 3:
tensor([[-0.9000,  2.9000]])
tensor([0])


In [22]:
# Train the neural network

import torch.nn.functional as F

torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)

# Count number of trainable parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_params} total parameters.')

optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 5

for epoch in range(num_epochs):
    model.train()

    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)

        loss = F.cross_entropy(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

         ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
        f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
        f" | Train Loss: {loss:.2f}")

model.eval()

with torch.no_grad():
    ou

Number of trainable parameters: 6
752 total parameters.
Epoch: 001/005 | Batch 000/003 | Train Loss: 0.75
Epoch: 001/005 | Batch 001/003 | Train Loss: 0.90
Epoch: 001/005 | Batch 002/003 | Train Loss: 0.65
Epoch: 002/005 | Batch 000/003 | Train Loss: 0.28
Epoch: 002/005 | Batch 001/003 | Train Loss: 0.92
Epoch: 002/005 | Batch 002/003 | Train Loss: 0.00
Epoch: 003/005 | Batch 000/003 | Train Loss: 0.04
Epoch: 003/005 | Batch 001/003 | Train Loss: 0.00
Epoch: 003/005 | Batch 002/003 | Train Loss: 0.32
Epoch: 004/005 | Batch 000/003 | Train Loss: 0.00
Epoch: 004/005 | Batch 001/003 | Train Loss: 0.01
Epoch: 004/005 | Batch 002/003 | Train Loss: 0.00
Epoch: 005/005 | Batch 000/003 | Train Loss: 0.00
Epoch: 005/005 | Batch 001/003 | Train Loss: 0.01
Epoch: 005/005 | Batch 002/003 | Train Loss: 0.00


NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=2, bias=True)
  )
)